In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/EECS549/Project/Final

/content/drive/MyDrive/EECS549/Project/Final


In [ ]:
import gc
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

#Data

##Import Dataset

In [ ]:
%%time
with open('mod_tft_match_info_NA1_12082020.json', 'r') as f:
    data_na = json.load(f)

with open('mod_tft_match_info_KR_12102020.json', 'r') as f:
    data_kr = json.load(f)
    
df = pd.DataFrame(columns=(['companion', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'puuid', 'time_eliminated', 'total_damage_to_players', 'traits', 'units']))

# ls = []
# for i in data_na:
#     for j in i['participants']:
#         ls.append(j)

# for i in data_kr:
#     for j in i['participants']:
#         ls.append(j)

data = df.append(data_na+data_kr)
data.drop(['companion', 'puuid'], axis=1, inplace=True)
print(data.shape)

data_na = []
data_kr = []
del data_na
del data_kr
gc.collect()

(1570096, 9)
CPU times: user 1min 2s, sys: 13.2 s, total: 1min 15s
Wall time: 1min 26s


##Load Static Data

In [ ]:
 # Champion data
def loadChampionsList():   
    with open('./tft_set4_static_data/champions.json') as f:
        champions = json.load(f)
    # 生成英雄ID列表    
    champions_id = []
    champions_cost = []
    for i in champions:
        champions_id.append(i['championId'])
        champions_cost.append(i['cost'])
    return champions_id, champions_cost

# Item data
def loadItemsList():
    with open('./tft_set4_static_data/items.json') as f:
        items = json.load(f)

    items_name = [i['name'] for i in items]
    items_id = [i['id'] for i in items]
    return items_name, items_id

champions_id, champions_cost = loadChampionsList()
items_name, items_id = loadItemsList()

#Machine Learning

##Input Output Transformation

In [ ]:
# Transform input to a uniform format for training
def transformInput(units, is_train=True):
    N = len(units) 
    X_tier = np.zeros((N, len(champions_id)))
    X_item = np.zeros((N, len(champions_id)))
    units = np.array(units)
    del_ls = []
    for i in range(N):
        if is_train:
            if len(units[i]) == 8:
                for j in units[i]:
                    X_tier[i, champions_id.index(j['character_id'])] = j['tier']
                    # match item count
                    if len(j['items']) > 0:
                        X_item[i, champions_id.index(j['character_id'])] = len(j['items'])
            else:
                del_ls.append(i)
        else:
            for j in units[i]:
                X_tier[i, champions_id.index(j['character_id'])] = j['tier']
                # match item count
                if len(j['items']) > 0:
                    X_item[i, champions_id.index(j['character_id'])] = len(j['items'])
            
    X_con = np.append(X_tier, X_item, axis=1).astype('int')
    if not is_train:
        return X_con
    X_con = np.delete(X_con, del_ls, axis=0)  
    return X_con, del_ls

In [ ]:
%%time
X, del_ls = transformInput(data['units'])
print(X.shape)

(961320, 116)
CPU times: user 12.7 s, sys: 2.94 s, total: 15.7 s
Wall time: 15.8 s


In [ ]:
# Converting placement to win rate [1-8] --> [100%, 0%]
def transformOutput(placement):
    placement = placement.astype(int)
    return np.delete(np.array((max(placement) - placement) / (max(placement) - min(placement))), del_ls)

(961320,)


In [ ]:
y = transformOutput(data['placement'])
print(y.shape)

##Split Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(672924, 116)
(672924,)
(288396, 116)
(288396,)


##Define Model

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(X.shape[1]),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               14976     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 23,297
Trainable params: 23,297
Non-trainable params: 0
_________________________________________________________________


##Train Model

In [ ]:
tf.random.set_seed(0)
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
10515/10515 [==============================] - 15s 1ms/step - loss: 0.0353 - accuracy: 0.1680
Epoch 2/10
10515/10515 [==============================] - 15s 1ms/step - loss: 0.0352 - accuracy: 0.1682
Epoch 3/10
10515/10515 [==============================] - 15s 1ms/step - loss: 0.0350 - accuracy: 0.1683
Epoch 4/10
10515/10515 [==============================] - 15s 1ms/step - loss: 0.0348 - accuracy: 0.1682
Epoch 5/10
10515/10515 [==============================] - 15s 1ms/step - loss: 0.0348 - accuracy: 0.1684
Epoch 6/10
10515/10515 [==============================] - 15s 1ms/step - loss: 0.0346 - accuracy: 0.1683
Epoch 7/10
10515/10515 [==============================] - 15s 1ms/step - loss: 0.0345 - accuracy: 0.1684
Epoch 8/10
10515/10515 [==============================] - 16s 1ms/step - loss: 0.0345 - accuracy: 0.1684
Epoch 9/10
10515/10515 [==============================] - 15s 1ms/step - loss: 0.0344 - accuracy: 0.1684
Epoch 10/10
10515/10515 [==============================

##Model Evaluation

In [ ]:
model.evaluate(X_test, y_test)

9013/9013 [==============================] - 9s 985us/step - loss: 0.0355 - accuracy: 0.1679


[0.035514701157808304, 0.16788721084594727]

#Test

In [ ]:
# Find placement 1 data index
cnt = 0
for i in range(data.shape[0]):
    if data.iloc[i, 3] == 1 and len(data.iloc[i,8])==8:
        print(i)
        cnt += 1

    if (cnt == 20):
        break

13
29
69
88
112
123
151
163
218
259
268
276
298
304
318
332
400
461
464
492


In [ ]:
# Example
print(data['placement'][268])
data['units'][268]

1


[{'character_id': 'TFT4_Jax',
  'chosen': 'Duelist',
  'items': [17, 35, 57],
  'name': '',
  'rarity': 1,
  'tier': 2},
 {'character_id': 'TFT4_Irelia',
  'items': [],
  'name': '',
  'rarity': 2,
  'tier': 1},
 {'character_id': 'TFT4_Lux', 'items': [], 'name': '', 'rarity': 2, 'tier': 2},
 {'character_id': 'TFT4_Warwick',
  'items': [24, 12, 69],
  'name': '',
  'rarity': 3,
  'tier': 2},
 {'character_id': 'TFT4_Ashe',
  'items': [18],
  'name': '',
  'rarity': 3,
  'tier': 2},
 {'character_id': 'TFT4_Shen',
  'items': [],
  'name': '',
  'rarity': 3,
  'tier': 2},
 {'character_id': 'TFT4_LeeSin',
  'items': [17, 69, 15],
  'name': '',
  'rarity': 4,
  'tier': 2},
 {'character_id': 'TFT4_Yone',
  'items': [],
  'name': '',
  'rarity': 4,
  'tier': 1}]

##Fill Combination

In [ ]:
def addNewChampions(test, items_num=0):
    ls = list(transformInput([test], is_train=False)[0]) # Format input
    arr_c = np.array([ls for i in range(len(champions_id))])
    for i in range(len(champions_id)):
        # Add champion according to current level
        if len(test) < 7:
            cost_limit = 5
        else:
            cost_limit = 6
        if arr_c[i,i] == 0 and champions_cost[i] < cost_limit:
            arr_c[i,i] = 2
            arr_c[i, i+len(champions_id)] = items_num
    return arr_c
    
def completeChampions(test, items_num=0):
    while len(test) < 8:
        win_rate_old = model.predict(transformInput([test], is_train=False))
        arr_c = addNewChampions(test, items_num)
        win_rate = model.predict(arr_c)
        index = np.where(win_rate == max(win_rate))[0][0]
        print('-----------------------')
        print('Champions num:', len(test))
        print('Existing champions:', [i['character_id'] for i in test])
        print('New champion:', champions_id[index])
        print('Win rate:', win_rate_old[0], '-->', max(win_rate))
        test.append({'character_id':champions_id[index], 'tier':2, 'items':[]})

##Make Recommendation

In [ ]:
test = data['units'][268][:6]
completeChampions(test, 1)

-----------------------
Champions num: 6
Existing champions: ['TFT4_Jax', 'TFT4_Irelia', 'TFT4_Lux', 'TFT4_Warwick', 'TFT4_Ashe', 'TFT4_Shen']
New champion: TFT4_LeeSin
Win rate: [0.21934527] --> [0.47836673]
-----------------------
Champions num: 7
Existing champions: ['TFT4_Jax', 'TFT4_Irelia', 'TFT4_Lux', 'TFT4_Warwick', 'TFT4_Ashe', 'TFT4_Shen', 'TFT4_LeeSin']
New champion: TFT4_Zilean
Win rate: [0.34950024] --> [0.6289367]


##Baseline Method

In [ ]:
import gensim

all_doc_list = []
for i in data[data['placement']==1]['units']:
    all_doc_list.append([j['character_id'] for j in i])
    
    
dictionary = gensim.corpora.Dictionary(all_doc_list)
corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]
index = gensim.similarities.SparseMatrixSimilarity(corpus, num_features=len(dictionary.keys()))

In [ ]:
def testToBow(test):
    ls = [i['character_id'] for i in test]
    return dictionary.doc2bow(ls)

def mostSimChampions(test):
    test_bow = testToBow(test)
    return np.array(data[data['placement']==1]['units'])[list(index[test_bow]).index(max(index[test_bow]))]

test = data['units'][268][:6]
mostSimChampions(test)

[{'character_id': 'TFT4_Jax',
  'items': [57, 77],
  'name': '',
  'rarity': 1,
  'tier': 2},
 {'character_id': 'TFT4_Janna',
  'items': [],
  'name': '',
  'rarity': 1,
  'tier': 2},
 {'character_id': 'TFT4_Irelia',
  'items': [],
  'name': '',
  'rarity': 2,
  'tier': 2},
 {'character_id': 'TFT4_Lux',
  'items': [33, 3],
  'name': '',
  'rarity': 2,
  'tier': 2},
 {'character_id': 'TFT4_Warwick',
  'items': [69, 49, 49],
  'name': '',
  'rarity': 3,
  'tier': 2},
 {'character_id': 'TFT4_Ashe',
  'chosen': 'Hunter',
  'items': [17, 29, 18],
  'name': '',
  'rarity': 3,
  'tier': 2},
 {'character_id': 'TFT4_Shen',
  'items': [],
  'name': '',
  'rarity': 3,
  'tier': 1}]